In [72]:
import numpy as np
from scipy.io import wavfile


# =======================================
# Parameters to tune for audio processing
# =======================================

g = 0.5 # Amplitude multiplier

# We use cartesian coordinates to model the motion of a point audio source
# around a listener centered around the origin

# The left ear is located at (-10, 0), the right ear at (10, 0)

left_ear_pos = np.array([-10, 0])
right_ear_os = np.array([10, 0])



'''
Adds a spatial characteristic to an input .wav audio file

param[in] input_path : path to the input audio file
param[out] output_path : paht to the output audio file
param[in] span : [0, 100] with 100 representing an audio signal coming from the right and 0 representing an audio signal
                coming from the left
'''
# TO DO: Change parameter to accept position of audio signal in space 
# Frequency attenuation 
# Hardcode left and right channel positions relative to origin

def head_transfer_function(input_path : str, output_path : str, span : int):
    span = 0 if span < 0 else min(span, 100)

    sample_rate, data = wavfile.read(input_path)
    audio_data = np.array(data, dtype=np.float32)

    if data.dtype == np.int16:
        audio_data = audio_data / 32768.0  # 16-bit audio normalization
    elif data.dtype == np.int32:
        audio_data = audio_data / 2147483648.0  # 32-bit audio normalization
    elif data.dtype == np.uint8:
        audio_data = (audio_data - 128) / 128.0  # 8-bit audio normalization

    sample_T = 1 / sample_rate # Period of sample (in s)

    # -100 = 0.0003
    # 0 = 0
    # 100 = 0.0003

    time_delay = 0.0003 # Time delay to imitate audio traveling from one ear to the other
    sample_shift = int(time_delay / sample_T) # Number of samples we have to shift by

    right_span = span
    left_span = 100 - span

    # Channel shifts are inversely proportional to their respective channel spans
    # thus directly proportional with the span of the opposite channel

    left_sample_shift = int(0.0003 * ((right_span) / 100) / sample_T)
    right_sample_shift = int(0.0003 * ((left_span) / 100) / sample_T)

    '''left_sample_shift = 0 if span < 100 else int(0.0003 * ((span - 100) / 100) / sample_T)
    right_sample_shift = 0 if span > 100 else int(0.0003 * ((100 - span) / 100) / sample_T)'''

    left_sample_padding = right_sample_shift
    right_sample_padding = left_sample_shift

    left_amp_mult = g + (1 - g) * (left_span / 100)
    right_amp_mult = g + (1 - g) * (right_span / 100)

    '''left_amp_mult = 1 if span <= 100 else (g * (100 / (span - 100)))
    right_amp_mult = 1 if span >= 100 else (g * (100 / (100 - span)))'''

    print(f"span : left shift : right shift | {span, left_sample_shift, right_sample_shift}")
    print(f"left_amp_mult {left_amp_mult}, right_amp_mult {right_amp_mult}")

    left_channel_padded = np.array([0. for _ in range(left_sample_shift)] + [audio_data[i] for i in range(len(audio_data))] + [0. for _ in range(left_sample_padding)], 
                                   dtype=np.float32)
    left_channel_padded *= left_amp_mult
    right_channel_padded = np.array([0. for _ in range(right_sample_shift)] + [audio_data[i] for i in range(len(audio_data))] + [0. for _ in range(right_sample_padding)], 
                                   dtype=np.float32)
    right_channel_padded *= right_amp_mult

    return (left_channel_padded, right_channel_padded, sample_rate)
    

def get_span_from_point(point : np.ndarray):
    point = np.array(point)
    #dist = np.linalg.norm(left_ear_pos - point)
    r_dist = np.linalg.norm(right_ear_os - point)
    l_dist = np.linalg.norm(left_ear_pos - point)

    diff = abs(r_dist - l_dist)
    diff = min(diff, 20)

    if r_dist <= l_dist:
        span = 50 + 50 * (diff / 20)
    else:
        span = 50 - (50 * (diff / 20))
    
    return span


In [70]:
def spatialize_from_point(input_path : str, output_path : str, point : list):
    try:
        _, _ = point
    except ValueError:
        print(f"ERROR: Point passed into spatialize_from_point is of invalid format. Expected (x, y). Received: {point}")
        return
    
    span = get_span_from_point(point)

    left_channel_padded, right_channel_padded, sample_rate = head_transfer_function(input_path, output_path, span)

    tone_y_stereo=np.vstack((left_channel_padded, right_channel_padded))
    tone_y_stereo=tone_y_stereo.transpose()
    wavfile.write(output_path, sample_rate, tone_y_stereo)


In [14]:
head_transfer_function("drum.wav", "output.wav", 100)
head_transfer_function("drum.wav", "left_output.wav", 0)
head_transfer_function("drum.wav", "middle.wav", 50)

NameError: name 'head_transfer_function' is not defined

In [12]:
head_transfer_function("drum.wav", "experiment.wav", 20)

span : left shift : right shift | (20, 2, 10)
left_amp_mult 0.9, right_amp_mult 0.6


In [73]:
spatialize_from_point("drum.wav", "experiment.wav", [100, 10])

span : left shift : right shift | (99.74939697624609, 13, 0)
left_amp_mult 0.5012530151187695, right_amp_mult 0.9987469848812305
